# Option Pricing Fixed Point and Newton Methods

In [4]:
from scipy.stats import norm
import numpy as np
import pandas as pd

## Part (a)

In [29]:
# import blsprice function
def blsprice(Price, Strike, Rate, Time, Volatility):
    sigma_sqrtT = Volatility * np.sqrt (Time)

    d1 = 1 / sigma_sqrtT * (np.log(Price / Strike) + (Rate + Volatility**2 / 2) * Time)
    d2 = d1 - sigma_sqrtT

    phi1 = norm.cdf(d1)
    phi2 = norm.cdf(d2)
    disc = np.exp (-Rate * Time)
    F    = Price * np.exp ((Rate) * Time)

    Call = disc * (F * phi1 - Strike * phi2)
    Put  = disc * (Strike * (1 - phi2) + F * (phi1 - 1))
    return Call, Put

# fixed point iteration scheme
def fixedpoint(Price, Strike, Rate, Time, Volatility, N, M):
    W = []
    R = []

    w_k = 0
    w_kp1 = N/(M+N)*blsprice(Price+(M/N)*w_k, Strike, Rate, Time, Volatility)[0]
    change = w_k - w_kp1
    W.append(w_k)
    W.append(w_kp1)
    R.append(change)

    while abs(change)>=10**(-8):
        w_k = w_kp1
        w_kp1 = N/(M+N)*blsprice(Price+(M/N)*w_k, Strike, Rate, Time, Volatility)[0]
        change = w_k - w_kp1
        W.append(w_kp1)
        R.append(change)
    return [W,R]

# parameters
sigma = 0.3
r = 0.04
T = 2
K = 100
S0 = 100
N = 30*10**6
M = 7*10**6

# outputs
W = fixedpoint(S0,K,r,T,sigma,N,M)[0]
R = fixedpoint(S0,K,r,T,sigma,N,M)[1]

# simple text table
W = ["%0.7e" % member for member in W]
R = ["-"] + ["%0.7e" % member for member in R]
table = {"W^k":W,
         "R^k":R}
df = pd.DataFrame(table)
print(df)

              W^k             R^k
0   0.0000000e+00               -
1   1.6443118e+01  -1.6443118e+01
2   1.8533340e+01  -2.0902218e+00
3   1.8806075e+01  -2.7273429e-01
4   1.8841775e+01  -3.5700105e-02
5   1.8846450e+01  -4.6749682e-03
6   1.8847062e+01  -6.1222533e-04
7   1.8847142e+01  -8.0176482e-05
8   1.8847153e+01  -1.0499850e-05
9   1.8847154e+01  -1.3750525e-06
10  1.8847154e+01  -1.8007582e-07
11  1.8847154e+01  -2.3582597e-08
12  1.8847154e+01  -3.0883598e-09


## Part (b)

In [32]:
# import blsdelta
def blsdelta(Price, Strike, Rate, Time, Volatility):
    d1 = 1 / (Volatility * np.sqrt(Time)) * (np.log (Price / Strike) + (Rate + Volatility**2 / 2) * Time)

    phi = norm.cdf(d1)

    CallDelta = phi
    PutDelta = phi - 1
    return CallDelta, PutDelta

# Newton's Method
def newton(Price, Strike, Rate, Time, Volatility, N, M):
    W = []
    R = []
    
    w_k = 0
    f = w_k - (N/(N+M))*blsprice(Price+(M/N)*w_k,Strike,Rate,Time,Volatility)[0]
    fprime = 1 - M/(N+M)*blsdelta(Price+(M/N)*w_k,Strike,Rate,Time,Volatility)[0]
    w_kp1 = w_k - (f/fprime)
    change = w_k - w_kp1

    W.append(w_k)
    W.append(w_kp1)
    R.append(change)

    while abs(change)>=10**(-8):
        w_k = w_kp1
        f = w_k - (N/(N+M))*blsprice(Price+(M/N)*w_k,Strike,Rate,Time,Volatility)[0]
        fprime = 1 - M/(N+M)*blsdelta(Price+(M/N)*w_k,Strike,Rate,Time,Volatility)[0]
        w_kp1 = w_k - (f/fprime)
        change = w_k - w_kp1

        W.append(w_kp1)
        R.append(change)
    return [W,R]

# parameters
sigma = 0.3
r = 0.04
T = 2
K = 100
S0 = 100
N = 30*10**6
M = 7*10**6

# outputs
W = newton(S0,K,r,T,sigma,N,M)[0]
R = newton(S0,K,r,T,sigma,N,M)[1]

# simple text table
W = ["%0.7e" % member for member in W]
R = ["-"] + ["%0.7e" % member for member in R]
table = {"W^k":W,
         "R^k":R}
df = pd.DataFrame(table)
print(df)

             W^k             R^k
0  0.0000000e+00               -
1  1.8771691e+01  -1.8771691e+01
2  1.8847153e+01  -7.5461665e-02
3  1.8847154e+01  -1.1486674e-06
4  1.8847154e+01   7.1054274e-15
